In [1]:

from utils import *
from keras.layers import Input,Dense,LSTM
import numpy as np
import keras,keras.backend as K
import emoji
import matplotlib.pyplot as plt
%matplotlib notebook


Using TensorFlow backend.


In [2]:
TRAIN_PATH = './data/emojify_data.csv'
TEST_PATH = './data/test_emoji.csv'

In [3]:
train_data , train_labels = read_csv(TRAIN_PATH)
test_data,test_labels = read_csv(TEST_PATH)

In [128]:
def view(l,see_):
    for i,sent in enumerate(l) :
        print(sent,emojize(see_[i]))


she got me a nice present ❤
will you be my valentine ❤
you brighten my day ❤
I hate u 😞
I want food 🍴


In [7]:
def create_sequences(batch_x,batch_y):
    max_len = batch_maxlen(batch_x)
    time_steps=max_len
    dim=100
    sequences=[]
    for sentence in batch_x:
        words = sentence.strip().lower().split()
        sequence = []
        padding = max_len - len(words)
        words = words + ['unkwnval']*padding
        #print(words)
        for word in words: 
            sequence.append(nlp(word).vector[:128])
        sequences.append(sequence)
    return np.array(sequences),batch_y

In [4]:
batches_x,batches_y = create_batches(train_data,train_labels)


## 183 training sentences

In [5]:
len(train_data)

183

## using word2vec vectors from spacy

In [6]:
import spacy
nlp=spacy.load('en')

In [151]:
def model_():
    
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=[None,128]))
    model.add(LSTM(128,return_sequences=True))
    model.add(keras.layers.BatchNormalization())
    model.add(LSTM(128,return_sequences=False))
    model.add(keras.layers.BatchNormalization())
    model.add(Dense(5,activation='softmax'))
    
    return model

In [116]:
Model = model_()
opt= keras.optimizers.adam(lr=0.001)
Model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
allbatch=[]
for batch_x,batch_y in zip(batches_x,batches_y):
    allbatch.append(create_sequences(batch_x,batch_y))

In [117]:
Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, None, 128)         131584    
_________________________________________________________________
batch_normalization_12 (Batc (None, None, 128)         512       
_________________________________________________________________
lstm_18 (LSTM)               (None, 128)               131584    
_________________________________________________________________
batch_normalization_13 (Batc (None, 128)               512       
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 645       
Total params: 264,837
Trainable params: 264,325
Non-trainable params: 512
_________________________________________________________________


## take the batch's maximum length as the number of time steps 

In [118]:
for epoch in range(8):
    for batches in allbatch:
        batch_x,batch_y =batches
        #print(batch_y)
        batch_y = batch_y.astype(np.float)
        Model.fit(batch_x,batch_y,shuffle=False)

Epoch 1/1
32/32 [==============================] - 3s 98ms/step - loss: 1.6026 - acc: 0.0938
Epoch 1/1
26/26 [==============================] - 0s 5ms/step - loss: 1.6029 - acc: 0.2692
Epoch 1/1
32/32 [==============================] - 0s 4ms/step - loss: 1.2998 - acc: 0.7812
Epoch 1/1
26/26 [==============================] - 0s 5ms/step - loss: 1.1879 - acc: 0.6154
Epoch 1/1
32/32 [==============================] - 0s 5ms/step - loss: 0.9095 - acc: 0.8750
Epoch 1/1
26/26 [==============================] - 0s 5ms/step - loss: 0.7190 - acc: 0.7308
Epoch 1/1
32/32 [==============================] - 0s 5ms/step - loss: 0.5200 - acc: 0.9062
Epoch 1/1
26/26 [==============================] - 0s 6ms/step - loss: 0.2758 - acc: 0.9615
Epoch 1/1
32/32 [==============================] - 0s 5ms/step - loss: 0.3185 - acc: 0.8750
Epoch 1/1
26/26 [==============================] - 0s 5ms/step - loss: 0.1671 - acc: 0.9231
Epoch 1/1
32/32 [==============================] - 0s 5ms/step - loss: 0.1630 -

In [119]:
batches_x,batches_y = create_batches(test_data,test_labels)

In [120]:
x=create_sequences(test_data,test_labels)

In [121]:
y_pred=np.argmax(Model.predict(x[0]),axis=1)

In [122]:
test_label=test_labels.astype(np.int)

In [123]:
print('TEST ACCURACY',sum(y_pred==test_label)/58)

TEST ACCURACY 0.965517241379


## test sentence emoji  predicted emoji

In [158]:
for i,sent in enumerate(test_data) :
    print(sent,emojize(test_label[i]),emojize(y_pred[i]))

I want to eat	 🍴 🍴
he did not answer	 😞 😞
he got a raise	 😄 😄
she got me a present	 ❤ ❤
ha ha ha it was so funny	 😄 😄
he is a good friend	 ❤ ❤
I am upset	 😞 😞
We had such a lovely dinner tonight	 ❤ ❤
where is the food	 🍴 🍴
Stop making this joke ha ha ha	 😄 😄
where is the ball	 ⚾ ⚾
work is hard	 😞 😞
This girl is messing with me	 😞 😞
are you serious ha ha	 😄 😄
Let us go play baseball	 ⚾ ⚾
This stupid grader is not working 	 😞 😞
work is horrible	 😞 😞
Congratulation for having a baby	 😄 😄
stop messing around	 😞 😞
any suggestions for dinner	 🍴 🍴
I love taking breaks	 ❤ ❤
you brighten my day	 😄 ❤
I boiled rice	 🍴 🍴
she is a bully	 😞 😞
Why are you feeling bad	 😞 😞
I am upset	 😞 😞
I worked during my birthday	 😞 😞
My grandmother is the love of my life	 ❤ ❤
enjoy your break	 😄 😄
valentine day is near	 ❤ ❤
I miss you so much	 ❤ ❤
throw the ball	 ⚾ ⚾
My life is so boring	 😞 😞
she said yes	 😄 😄
will you be my valentine	 ❤ ❤
he can pitch really well	 ⚾ ⚾
dance with me	 😄 😄
I am starving	 🍴 🍴
See you

## LET'S TRY OWN DATA 

In [148]:
random_sentences = ['This world is a hard place to live',
                    'He is a weak boy',
                    'What a wonderful scenery',
                    'Nice to meet you',
                    'What do you want for food',
                    'I adore his kindness',
                    'I am going to play cricket',
                    'He won the match']

In [149]:
predictions = np.argmax(Model.predict(create_sequence(random_sentences)),axis=1)
print(predictions)

[0 0 2 0 4 2 0 1]


In [150]:
predictions = np.argmax(Model.predict(create_sequence(random_sentences)),axis=1)
print(predictions)
view(random_sentences,predictions)

[0 0 2 0 4 2 0 1]
This world is a hard place to live ❤
He is a weak boy ❤
What a wonderful scenery 😄
Nice to meet you ❤
What do you want for food 🍴
I adore his kindness 😄
I am going to play cricket ❤
He won the match ⚾
